## Convolutional Neural Networks

Although you can implement a neural network using only numpy library,. with adding more components to the network (activations, regularization, batchnorm, etc.), the code becomes very complex and difficult to manage. Using Pytorch allows us to modularize the components of a network and hide the complexity. 

We will build a neural network using PyTorch for FashionMnist data for Image classification. However, it would be impractical to build a deep neural network with only fully connected layers for images. Instead, we will build a Convolutional Neural Network, which is the most popular approach to extracting features from an image without too much computation (compared to a fully connected layer). 

**It will be ideal to solve this assignemnet on a computer with a GPU**. I personally used [Google Colab](https://www.youtube.com/watch?v=inN8seMm7UI&ab_channel=TensorFlow), but [Kaggle](https://www.kaggle.com/dansbecker/running-kaggle-kernels-with-a-gpu) is also an option.


We first import the neccesary packages required to build the model. We also fix a seed through out the process to have a deterministic behaviour (reproduce same results) of the model. 

In [ ]:
## Imports
import os
import time
import numpy as np
import torch
import random
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy.testing as npt
#from torchsummary import summary
# from tqdm import trange

# Checks for the availability of GPU 
is_cuda = torch.cuda.is_available()
if torch.cuda.is_available():
    print("working on gpu!")
else:
    print("No gpu! only cpu ;)")
    
## The following random seeds are just for deterministic behaviour of the code and evaluation

##############################################################################
################### DO NOT MODIFY THE CODE BELOW #############################    
##############################################################################

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = '0'

############################################################################### 


### STEP0. Setting up the DataLoaders

In the following cell we will first download and arrange the data. FashionMNIST dataset is already available in the official PyTorch repository. Hence, the following cell checks for the availability of FashionMNIST data and downloads if the data is not available.   

The following parts are already written for you to handle the data.
- import neccesary pytorch packages for data handling.
- We then move the data onto PyTorch tensors.  
- Next we define the parameters like batch_size for data handling. A different batch_size for test data is used to     make sure that number of samples in the test data are perfectly divisible.
- create dataloaders for training and testing data to iterate.

In [ ]:
import torchvision
import torchvision.transforms as transforms
import os

if not os.path.isdir('./data'):
    os.mkdir('./data')
root = './data/'

# List of transformation on the data - here we will normalize the image data to (-1,1)
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5), (0.5)),])
# Geta  handle to Load the data
training_data = torchvision.datasets.FashionMNIST(root, train=True, transform=transform,download=True)
testing_data = torchvision.datasets.FashionMNIST(root, train=False, transform=transform,download=True)

num_train = len(training_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(0.2 * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_bs = 60
test_bs = 50

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# Create Data loaders which we will use to extract minibatches of data to input to the network for training
train_loader = torch.utils.data.DataLoader(training_data, batch_size=train_bs,
    sampler=train_sampler, drop_last=False)
valid_loader = torch.utils.data.DataLoader(training_data, batch_size=train_bs, 
    sampler=valid_sampler, drop_last=False)
test_loader = torch.utils.data.DataLoader(testing_data, batch_size=test_bs, 
    drop_last=False)


### Visualize a Few Data Samples

In the following cell we first peek into a random batch of images together with labels and visualize them. 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

## get a batch of data
images, labels = iter(train_loader).next()


image_dict = {0:'T-shirt/Top', 1:'Trouser', 2:'Pullover', 3:'Dress',
              4:'Coat', 5:'Sandal', 6:'Shirt', 7:'Sneaker',
              8:'Bag', 9:'Ankle Boot'}

fig = plt.figure(figsize=(8,8))

print(images.size())

for i in np.arange(1, 13):
    ax = fig.add_subplot(3,4,i, frameon=False)
    img = images[i][0]
    ax.set_title(image_dict[labels[i].item()])
    plt.imshow(img, cmap='gray')


### STEP1. Definting the Architecture and Building the Model

We implement a Convolutional Neural Network as our model. We make use of the following layers in our model.
- a convolution layer for extracting features.
- batchnorm layer for normalizing the weights in the hidden layers.
- ReLU activation function for the non-linearity between layers.
- Finally fully connected layers in the end.

#### Model:

we make use of the following convolutional neural network architecture for our dataset. 

- convolution layer output_channels-16 kernel_size=3 stride=1 padding-1
- batchnormalization layer
- ReLU activation layer
- maxpool layer kernel_size=2 stride=2
- convolution layer output_channels-32 kernel_size=3 stride=1 padding-1
- batchnormalization layer
- ReLU activation layer
- maxpool layer kernel_size=2 stride=2
- convolution layer output_channels-64 kernel_size=5 stride=1 padding-2
- batchnormalization layer
- ReLU activation layer
- maxpool layer kernel_size=2 stride=2
- fully connected layer - number_of_classes



- We first define a class called Model nheriting from Pytorch's nn.Module.
- In init(constructor), we define all the layers that are used to build the model
- Define a forward function for a sequential model that takes in images as input and returns the predictions as output.

All the functions are available in the PyTorch package. Read the documentation/source code for a better understanding.

- Convolutional layer: https://pytorch.org/docs/stable/nn.html#convolution-layers
- Batchnorm layer: https://pytorch.org/docs/stable/nn.html#normalization-layers
- Activation ReLU: https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity
- Maxpooling layer: https://pytorch.org/docs/stable/nn.html#pooling-layers
- Fully connected layer: https://pytorch.org/docs/stable/nn.html#linear-layers



In [ ]:
import torch.nn as nn

class Model(nn.Module):
    ## init function is the constructor and we define all the layers used in our model. 
    def __init__(self, num_classes=10):
        super(Model, self).__init__()
         # define a convolutional layer with 16 channels, kernel_size=3, stride=1 and padding=1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride = 1, padding=1)

        # define a batchnormalization layer 
        self.batch1 = nn.BatchNorm2d(16)

        # define a relu layer
        self.relu1 = nn.ReLU()

        # define a maxpool layer with kernel_size=2, stride=2
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride = 2)
        
        # define a convolutional layer with 32 channels, kernel_size=3, stride=1 and padding=1
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride = 1, padding=1)
        
        # define a batchnormalization layer 
        self.batch2 = nn.BatchNorm2d(32)
        
        # define a relu layer
        self.relu2 = nn.ReLU()

        # define a maxpool layer with kernel_size=2, stride=2
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride = 2)
        
        # define a convolution layer with 64 channels, kernel_size=5, stride=1 and padding=2
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride = 1, padding=2)
              
        # define a batchnorm layer        
        self.batch3 = nn.BatchNorm2d(64)

        # define a relu layer
        self.relu3 = nn.ReLU()

        # define a maxpool layer with kernel_size=2, stride=2
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride = 2)

        # define a fully connected layer from resulting dimension -> number of classes
        
        self.fc = nn.Linear(64*3*3, num_classes)       
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        
        # Conv 1
        # We will start with feeding the data to the first layer. 
        
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.relu1(x) 
        # We take the output x and feed it back to the next layer 
        
        # Max Pool 1
        x = self.pool1(x)
        
        # Conv 2
        x = self.conv2(x)
        x = self.batch2(x)
        x = self.relu2(x)
        
        # Max Pool 2
        x = self.pool2(x)

        # Conv 3
        x = self.conv3(x)
        x = self.batch3(x)
        x = self.relu3(x)
        
        # Max Pool 3
        x = self.pool3(x)     
        
    # Note that before the fully connected(fc) layer, the output is a feature map with size (N,C,H,W)
    # but a fully connected layers expects a input of size (N, dimension). Therefore, before passing the output of
    # maxpool layer to the fc layer, we must first flatten the output of previous layer to a size (N,C*H*W) 
    # and then pass it to the fully connected layer

        x = x.flatten(x)
        
        # Linear Layer
        x = self.fc(x)        
        
        return x        
        
    
    # Note that before the fully connected(fc) layer, the output is a feature map with size (N,C,H,W)
    # but a fully connected layers expects a input of size (N, dimension). Therefore, before passing the output of
    # maxpool layer to the fc layer, we must first flatten the output of previous layer to a size (N,C*H*W) 
    # and then pass it to the fully connected layer, like x = x.flatten(x)
    def flatten(self, x):
        N, C, H, W = x.size()
        x = x.reshape((N, C*H*W))                
        
        return x
        
        
    

In [ ]:
# Setting up a few learning parameters
learning_rate = 1e-2
decayRate = 0.999
epochs = 5
number_of_classes = 10

### STEP2. Initialize the CNN Model

Define a loss criterion, In this assignment we will use cross-entropy loss between the predictions and ground truth to estimate the loss. 
- CrossEntropyLoss - https://pytorch.org/docs/stable/nn.html#crossentropyloss

We also define a optimization strategy to update the weights. In this assignment we use the most commonly used Adam optimizer from the PyTorch package.

- Adam - https://pytorch.org/docs/stable/optim.html#algorithms 

In [ ]:
## First we will define an instance of the model to train
model = Model(num_classes=number_of_classes)
print(model)

#Move the model to the gpu if is_cuda
if is_cuda:
  model = model.cuda()

# define the loss 'criterion' as nn.CrossEntropyLoss() object
criterion = nn.CrossEntropyLoss()


# Initialize the Adam optimizer for the model.parameters() using the learning_rate
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

# This is the learning rate scheduler. It decreases the learning rate as we approach convergence
my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)



### STEP3. Training the Model

The training loop is setup in the following way:

For every batch in the defined number of epochs

- Move the images and labels to the gpu by checking is_cuda
- Extract output by passing images through the model 
- pass the output and ground truth to the loss criterion for batch loss
- clear the gradients 
- backpropagate (compute gradients w.r.t the parameters) using backward() 
- update the parameters with a single optimization step
- update the training loss for plots

repeat

In [ ]:
## training loop 

## Number of epochs the model runs

def train_model(epochs=25, validate=True):
    '''
    A function to train the model on the dataset and returns the trained model, training loss and
    validation loss for every epoch.
    
    Inputs:
        epochs: Number of times the model should be trained on the whole data.
        validate: A boolean parameter that validates on validation data.
        
    Outputs:
        model: The model trained for specified number of epochs
        training loss: A list of training losses computed for every epoch.
        validation loss: A list of validation losses computed for every epoch.
    
    '''
    train_loss = []
    val_loss = []

    for epoch in range(epochs):
        # Iterate through the batches in the data
        training_loss = 0.0
        validation_loss = 0.0
        
        model.train()
        itr = 0
        for (images,labels)  in train_loader:
            # Move the images and labels to the gpu by checking is_cuda
            if is_cuda:
              images = images.cuda()
              labels = labels.cuda()
            # Extract outputs by passing images through the model
            outputs = model(images)
            
  
            # estimate loss using criterion(.) with 'outputs' and 'labels'
            loss = criterion(outputs, labels)
            # clear the gradients with .zero_grad()
            optimizer.zero_grad()
            # Estimate gradients using .backward()
            loss.backward()
            # update model with .step()
            optimizer.step()
            # update learning_rate with with a .step using my_lr_scheduler
            my_lr_scheduler.step()

            # add loss to training_loss
            training_loss += loss  
            
            if itr%100 == 0:
                print('Epoch %d/%d, itr = %d, Train Loss = %.3f, LR = %.3E'\
                      %(epoch, epochs, itr, loss.item(),optimizer.param_groups[0]['lr']))
            itr += 1
        train_loss.append(training_loss/len(train_loader))
        print('------------------------------------------------')
        
        if validate:
            model.eval()
            with torch.no_grad():
                itr = 0
                for (images,labels)  in valid_loader:
                    # During validation we do not update gradients. We set model to eval()
                    model.eval()
                    # Move images and labels to gpu if is_cuda
                    if is_cuda:
                      images = images.cuda()
                      labels = labels.cuda()
                    # get 'outputs' using model and images
                    outputs = model(images)
                    # estimate loss using criterion(.) with 'outputs' and 'labels'
                    loss = criterion(outputs,labels)
                    # add loss to 'validation_loss'
                    validation_loss += loss                           
                    if itr%100 == 0:
                        print('Epoch %d/%d, itr = %d, Val Loss = %.3f, LR = %.3E'\
                              %(epoch, epochs, itr, loss.item(),optimizer.param_groups[0]['lr']))
                    itr += 1
                val_loss.append(validation_loss/len(valid_loader))
                print('################################################')
                
    return model, train_loss, val_loss
                

In [ ]:
start = time.time()
trained_model, train_loss, val_loss = train_model(epochs, validate=True)
end = time.time()
print('Time to train in seconds ',(end - start))

In [ ]:
# Plot the losses
it = np.arange(epochs)
plt.plot(it, train_loss, label='training loss')
plt.plot(it, val_loss, label='validation loss')
plt.xlabel('epochs')
plt.ylabel('losses')
plt.legend(loc='upper right')
plt.show()

### STEP4. Testing the Classification

In the testing loop we don't update the weights. The trained model is tested for all the samples in test data to compute the accuracy and observe how well the model is generalizing to the unseen data. 

The testing loop is setup in the following way: 

For every batch in the testing data

- Put the model in the evaluation mode and turn off the gradients
- Move the images and labels to the device available
- extract output from the model for the input
- compute the prediction class by choosing the one with maximum probability in the predictions.
- Compare the prediction classes with true classes.
- calculate accuracy
- update test_loss for plots

repeat


In [ ]:
## Testing Loop

def evaluate_model(model, loader):
    '''
    A function to test the trained model on the dataset and print the accuracy on the testset.
    
    Inputs:
        model: Trained model
        loader: train_loader or test_loader
        
    outputs:
        accuracy. returns the accuracy of prediction
    '''
    model.eval()
    with torch.no_grad():
        correct = 0
        total_samples = 0
        for images, labels in loader:
            ## Move the images and labels to gpu if is_cuda 
            if is_cuda:
              images = images.cuda()
              labels = labels.cuda()
            
            ## Get the output of the model by passing images as input to the model
            preds = []
            outputs = model(images)
            ## estimate the index of the highest output in each row, that is the predicted label 
            preds.extend(outputs.argmax(axis=1).cpu().numpy())

            ## There is no need for softmax if the goal is to merely identify the predicted category

            ## compare predictions with ground truth for number of correct samples
            correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()
            
            total_samples += labels.size(0)
        
        accuracy = correct/total_samples*100
        print("Total Accuracy on the Input set: {} %".format(accuracy))
        return accuracy

In [ ]:
# With these settings, obtained 95% train and 91% test accuracy
tr_acc = evaluate_model(model, train_loader)
ts_acc = evaluate_model(model, test_loader)
print('Train Accuracy = %.3f'%(tr_acc))
print('Test Accuracy = %.3f'%(ts_acc))

In [ ]:
## Visualize the test samples with predicted output and true output
images, labels = iter(test_loader).next()
# images = images.numpy()
if is_cuda:
  images = images.cuda()
  labels = labels.cuda()

out = model(images)
_, preds = torch.max(out, dim=1)

images = images.to('cpu').numpy()

fig = plt.figure(figsize=(15,15))
for i in np.arange(1, 13):
    ax = fig.add_subplot(4, 3, i)
    plt.imshow(images[i][0])
    ax.set_title("Predicted: {}/ Actual: {}".format(image_dict[preds[i].item()], image_dict[labels[i].item()]), 
                color=('green' if preds[i] == labels[i] else 'red'))